# 🔁 Experience Replay

Uma grande desvantagem das redes neurais é a necessidade de treinar com uma grande quantidade de dados para obter um bom aprendizado. Isso torna seu uso em algoritmos "online" como os de Temporal Difference bem difícil, já que ela recebe apenas uma transição a cada instante de tempo para o treinamento.

Entretanto, como Q-Learning é um algoritmo off-policy, nós podemos aproveitar as experiências anteriores do nosso agente para utilizar em um batch no treinamento da nossa rede. É dessa ideia que surge o conceito do **Experience Replay**, um buffer para guardar todas as experiências passadas do nosso agente.

Para entender como isso funciona, vamos relembrar da atualização do Q-Learning:

$$Q(S, A) \leftarrow Q(S, A) + \alpha [R + \gamma \max_{a}Q(S', a) - Q(S, A)]$$

Observe que para atualizar o valor *Q* de um par estado-ação, precisamos saber apenas o estado *S*, a ação tomada *A*, a recompensa recebida *R* e o estado seguinte *S'*. Como esse update não depende da política no momento da escolha da ação, podemos usar uma experiência $(s_t, a_t, r_t, s_{t+1})$ para treinamento a qualquer momento.

Dessa forma, o que podemos fazer é guardar esses pares $(s_t, a_t, r_t, s_{t+1})$ em um buffer, e amostrar uma batch dessas experiências passadas para cada treinamento. Assim, conseguimos reaproveitar as experiências obtidas pelo nosso agente e aumentar a *sample efficiency* do nosso algoritmo, ou seja, sua eficiência dado uma quantidade limitada de experiências.

A seguir, segue uma implementação desse Buffer de experiências:

In [ ]:
import numpy as np

class ReplayBuffer:
    """Experience Replay Buffer para DQNs."""
    def __init__(self, max_length, observation_space):
        """Cria um Replay Buffer.

        Parâmetros
        ----------
        max_length: int
            Tamanho máximo do Replay Buffer.
        observation_space: int
            Tamanho do espaço de observação.
        """
        self.index, self.size, self.max_length = 0, 0, max_length

        self.states = np.zeros((max_length, observation_space), dtype=np.float32)
        self.actions = np.zeros((max_length), dtype=np.int32)
        self.rewards = np.zeros((max_length), dtype=np.float32)
        self.next_states = np.zeros((max_length, observation_space), dtype=np.float32)
        self.dones = np.zeros((max_length), dtype=np.float32)

    def __len__(self):
        """Retorna o tamanho do buffer."""
        return self.size

    def update(self, state, action, reward, next_state, done):
        """Adiciona uma experiência ao Replay Buffer.

        Parâmetros
        ----------
        state: np.array
            Estado da transição.
        action: int
            Ação tomada.
        reward: float
            Recompensa recebida.
        state: np.array
            Estado seguinte.
        done: int
            Flag indicando se o episódio acabou.
        """
        self.states[self.index] = state
        self.actions[self.index] = action
        self.rewards[self.index] = reward
        self.next_states[self.index] = next_state
        self.dones[self.index] = done
        
        self.index = (self.index + 1) % self.max_length
        if self.size < self.max_length:
            self.size += 1
            
    def sample(self, batch_size):
        """Retorna um batch de experiências.
        
        Parâmetros
        ----------
        batch_size: int
            Tamanho do batch de experiências.

        Retorna
        -------
        states: np.array
            Batch de estados.
        actions: np.array
            Batch de ações.
        rewards: np.array
            Batch de recompensas.
        next_states: np.array
            Batch de estados seguintes.
        dones: np.array
            Batch de flags indicando se o episódio acabou.
        """
        # Escolhe índices aleatoriamente do Replay Buffer
        idxs = np.random.randint(0, self.size, size=batch_size)

        return (self.states[idxs], self.actions[idxs], self.rewards[idxs], self.next_states[idxs], self.dones[idxs])